# Flow Theory

# IMPORTS

In [ ]:
from typing import List, Tuple
import math
from collections import Counter, deque, defaultdict
import urllib.request
import time

# UTILS

In [ ]:
def conv_seconds_milliseconds(seconds: float) -> float:
    return seconds * 1000

def duration(start: float, end: float) -> float:
    return end - start


# Max Flow

## FORD FULKERSON ALGORITHM

In [ ]:
"""
FordFulkeron algorithm for maximum flow problem 
- pluggable augmenting path finding algorithms
- residual graph
- bottleneck capacity
- merge parallele edges by sum of all parallel edges capacities
- depth first search
- Edmonds-Karp algorithm
- capacity scaling algorithm
- dinics algorithm
"""
class FordFulkersonMaxFlowV1:
    """
    Ford-Fulkerson algorithm 
    - pluggable augmenting path finding algorithms
    - residual graph
    - bottleneck capacity
    """
    def __init__(self, n: int, edges: List[Tuple[int, int, int]]):
        self.size = n
        self.edges = edges

    def build(self, n: int, edges: List[Tuple[int, int, int]]) -> None:
        self.adj_list = {}
        self.delta = cnt = 0
        for u, v, cap in edges:
            if u not in self.adj_list:
                self.adj_list[u] = Counter()
            self.adj_list[u][v] += cap
            if v not in self.adj_list:
                self.adj_list[v] = Counter()
            self.delta = max(self.delta, self.adj_list[u][v])
        highest_bit_set = self.delta.bit_length() - 1
        self.delta = 1 << highest_bit_set

    def main_dfs(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            cur_flow = self.dfs(source, sink, math.inf)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def reset(self) -> None:
        self.parents = [-1] * self.size

    def dfs(self, node: int, sink: int, flow: int) -> int:
        if node == sink:
            return flow
        self.parents[node] = 1
        cap = self.adj_list[node]
        for nei, cap in cap.items():
            if self.parents[nei] == -1 and cap > 0:
                cur_flow = self.dfs(nei, sink, min(flow, cap))
                if cur_flow > 0:
                    self.adj_list[node][nei] -= cur_flow
                    self.adj_list[nei][node] += cur_flow
                    return cur_flow
        return 0
    
    def main_edmonds_karp(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            cur_flow = self.edmonds_karp(source, sink)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def edmonds_karp(self, source: int, sink: int) -> int:
        queue = deque([(source, math.inf)])
        self.parents[source] = -2
        while queue:
            node, flow = queue.popleft()
            if node == sink:
                break
            capacity = self.adj_list[node]
            for nei, cap in capacity.items():
                if self.parents[nei] == -1 and cap > 0:
                    self.parents[nei] = node
                    queue.append((nei, min(flow, cap)))
        if node == sink:
            while node != source:
                parent = self.parents[node]
                self.adj_list[parent][node] -= flow
                self.adj_list[node][parent] += flow # residual edge
                node = parent
            return flow
        return 0

    def main_capacity_scaling(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while self.delta > 0:
            while True:
                self.reset()
                cur_flow = self.capacity_scaling(source, sink, math.inf)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
            self.delta >>= 1
        return maxflow

    def capacity_scaling(self, source: int, sink: int, flow: int) -> int:
        if source == sink:
            return flow
        self.parents[source] = 1
        capacity = self.adj_list[source]
        for nei, cap in capacity.items():
            if self.parents[nei] == -1 and cap >= self.delta:
                cur_flow = self.capacity_scaling(nei, sink, min(flow, cap))
                if cur_flow > 0:
                    self.adj_list[source][nei] -= cur_flow
                    self.adj_list[nei][source] += cur_flow
                    return cur_flow
        return 0

    def dinics_bfs(self, source: int, sink: int) -> bool:
        self.distances = [-1] * self.size
        self.distances[source] = 0
        queue = deque([source])
        while queue:
            node = queue.popleft()
            for nei, cap in self.adj_list[node].items():
                if self.distances[nei] == -1 and cap > 0:
                    self.distances[nei] = self.distances[node] + 1
                    queue.append(nei)
        return self.distances[sink] != -1

    def dinics_dfs(self, node: int, sink: int, flow: int) -> int:
        if flow == 0: return 0
        if node == sink: return flow
        for nei, cap in self.adj_list[node].items():
            if self.distances[nei] == self.distances[node] + 1 and cap > 0:
                cur_flow = self.dinics_dfs(nei, sink, min(flow, cap))
                if cur_flow > 0:
                    self.adj_list[node][nei] -= cur_flow
                    self.adj_list[nei][node] += cur_flow
                    return cur_flow
        return 0

    def main_dinics(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while self.dinics_bfs(source, sink):
            self.reset()
            while True:
                cur_flow = self.dinics_dfs(source, sink, math.inf)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
        return maxflow

In [ ]:
"""
FordFulkeron algorithm for maximum flow problem 
- pluggable augmenting path finding algorithms
- residual graph
- bottleneck capacity
- flow edge class
- depth first search
- Edmonds-Karp algorithm
- capacity scaling algorithm
- dinics algorithm
-- deadend elimination
"""
class FlowEdge:
    def __init__(self, src: int, dst: int, cap: int):
        self.src = src # source node
        self.dst = dst # destination node
        self.cap = cap

    def __repr__(self):
        return f'source node: {self.src}, destination node: {self.dst}, capacity: {self.cap} ======'

class FordFulkersonMaxFlowV2:
    """
    Ford-Fulkerson algorithm 
    - pluggable augmenting path finding algorithms
    - residual graph
    - bottleneck capacity
    """
    def __init__(self, n: int, edges: List[Tuple[int, int, int]]):
        self.size = n
        self.edges = edges

    def build(self, n: int, edges: List[Tuple[int, int, int]]) -> None:
        self.flowedges = []
        self.adj_list = {}
        self.delta = 0
        for u, v, cap in edges:
            self.flowedges.append(FlowEdge(u, v, cap))
            if u not in self.adj_list:
                self.adj_list[u] = []
            self.adj_list[u].append(len(self.flowedges) - 1)
            self.flowedges.append(FlowEdge(v, u, 0)) # residual edge
            if v not in self.adj_list:
                self.adj_list[v] = []
            self.adj_list[v].append(len(self.flowedges) - 1)
            self.delta = max(self.delta, cap)
        highest_bit_set = self.delta.bit_length() - 1
        self.delta = 1 << highest_bit_set

    def main_dfs(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            cur_flow = self.dfs(source, sink, math.inf)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def reset(self) -> None:
        self.vis = [0] * self.size

    def neighborhood(self, node: int) -> List[int]:
        return (i for i in self.adj_list[node])

    def dfs(self, node: int, sink: int, flow: int) -> int:
        if node == sink:
            return flow
        self.vis[node] = 1
        for index in self.neighborhood(node):
            nei = self.flowedges[index]
            if self.vis[nei.dst] == 0 and nei.cap > 0:
                cur_flow = self.dfs(nei.dst, sink, min(flow, nei.cap))
                if cur_flow > 0:
                    nei.cap -= cur_flow
                    self.flowedges[index ^ 1].cap += cur_flow
                    return cur_flow
        return 0
    
    def main_edmonds_karp(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            self.parents = [-1] * len(self.flowedges)
            cur_flow = self.edmonds_karp(source, sink)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def edmonds_karp(self, source: int, sink: int) -> int:
        queue = deque([(source, math.inf, -1)])
        self.vis[source] = 1
        while queue:
            node, flow, prev_index = queue.popleft()
            if node == sink:
                break
            for index in self.neighborhood(node):
                nei = self.flowedges[index]
                if self.vis[nei.dst] == 0 and nei.cap > 0:
                    self.vis[nei.dst] = 1
                    self.parents[index] = prev_index
                    queue.append((nei.dst, min(flow, nei.cap), index))
        if node == sink:
            while prev_index != -1:
                parent_index = self.parents[prev_index]
                self.flowedges[prev_index].cap -= flow
                self.flowedges[prev_index^1].cap += flow # residual edge
                prev_index = parent_index
            return flow
        return 0

    def main_capacity_scaling(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while self.delta > 0:
            while True:
                self.reset()
                cur_flow = self.capacity_scaling(source, sink, math.inf)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
            self.delta >>= 1
        return maxflow

    def capacity_scaling(self, node: int, sink: int, flow: int) -> int:
        if node == sink:
            return flow
        self.vis[node] = 1
        for index in self.neighborhood(node):
            nei = self.flowedges[index]
            if self.vis[nei.dst] == 0 and nei.cap >= self.delta:
                cur_flow = self.capacity_scaling(nei.dst, sink, min(flow, nei.cap))
                if cur_flow > 0:
                    nei.cap -= cur_flow
                    self.flowedges[index ^ 1].cap += cur_flow
                    return cur_flow
        return 0

    def dinics_bfs(self, source: int, sink: int) -> bool:
        self.distances = [-1] * self.size
        self.distances[source] = 0
        queue = deque([source])
        while queue:
            node = queue.popleft()
            for index in self.neighborhood(node):
                nei = self.flowedges[index]
                if self.distances[nei.dst] == -1 and nei.cap > 0:
                    self.distances[nei.dst] = self.distances[node] + 1
                    queue.append(nei.dst)
        return self.distances[sink] != -1

    def dinics_dfs(self, node: int, sink: int, flow: int) -> int:
        if flow == 0: return 0
        if node == sink: return flow
        while self.ptr[node] < len(self.adj_list[node]):
            index = self.adj_list[node][self.ptr[node]]
            self.ptr[node] += 1
            nei = self.flowedges[index]
            if self.distances[nei.dst] == self.distances[node] + 1 and nei.cap > 0:
                cur_flow = self.dinics_dfs(nei.dst, sink, min(flow, nei.cap))
                if cur_flow > 0:
                    nei.cap -= cur_flow
                    self.flowedges[index ^ 1].cap += cur_flow
                    return cur_flow
        return 0

    def main_dinics(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while self.dinics_bfs(source, sink):
            self.reset()
            self.ptr = [0] * self.size # pointer to the next edge to be processed (optimizes for dead ends)
            while True:
                cur_flow = self.dinics_dfs(source, sink, math.inf)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
        return maxflow

## PUSH RELABEL ALGORITHM

In [ ]:
"""
Push Relabel Algorithm
- current-arc to avoid iterating over all the edges each time in the discharge operation
- way to slow to be feasible
"""
class PushRelabelV1:
    def __init__(self, size: int, edges: List[Tuple[int, int, int]]):
        self.size = size
        self.edges = edges
        self.height = [0] * size
        self.excess = [0] * size
        self.seen = [0] * self.size
        self.cap = [[0] * size for _ in range(size)]
        self.flow = [[0] * size for _ in range(size)]
        self.excess_queue = deque()

    def remaining_capacity(self, src: int, dst: int) -> int:
        return self.cap[src][dst] - self.flow[src][dst]

    def build(self):
        for src, dst, cap in self.edges:
            self.cap[src][dst] += cap 

    def push(self, src: int, dst: int) -> None:
        flow = min(self.excess[src], self.remaining_capacity(src, dst))
        self.excess[src] -= flow
        self.excess[dst] += flow
        self.flow[src][dst] += flow
        self.flow[dst][src] -= flow
        if flow > 0 and self.excess[dst] == flow:
            self.excess_queue.append(dst)

    def relabel(self, node: int) -> None:
        minh = math.inf
        for nei in range(self.size):
            if self.remaining_capacity(node, nei) > 0:
                minh = min(minh, self.height[nei])
        if minh < math.inf:
            self.height[node] = minh + 1
    
    def discharge(self, node: int) -> None:
        while self.excess[node] > 0:
            if self.seen[node] < self.size:
                nei = self.seen[node]
                if self.remaining_capacity(node, nei) > 0 and self.height[node] > self.height[nei]:
                    self.push(node, nei)
                else:
                    self.seen[node] += 1
            else:
                self.relabel(node)
                self.seen[node] = 0
    
    def main(self, source: int, sink: int) -> int:
        self.build()
        self.height[source] = self.size
        self.excess[source] = math.inf
        for nei in range(self.size):
            if nei == source: continue
            self.push(source, nei)
        while self.excess_queue:
            node = self.excess_queue.popleft()
            if node != source and node != sink:
                self.discharge(node)
            
        return sum(self.flow[source])

In [ ]:
"""
Push Relabel Algorithm
- greatest height optimization
- way to slow to be feasible some reason
"""
class PushRelabelV2:
    def __init__(self, size: int, edges: List[Tuple[int, int, int]]):
        self.size = size
        self.edges = edges
        self.height = [0] * size
        self.excess = [0] * size
        self.seen = [0] * self.size
        self.cap = [[0] * size for _ in range(size)]
        self.flow = [[0] * size for _ in range(size)]

    def remaining_capacity(self, src: int, dst: int) -> int:
        return self.cap[src][dst] - self.flow[src][dst]

    def build(self):
        for src, dst, cap in self.edges:
            self.cap[src][dst] += cap 

    def push(self, src: int, dst: int) -> None:
        flow = min(self.excess[src], self.remaining_capacity(src, dst))
        self.excess[src] -= flow
        self.excess[dst] += flow
        self.flow[src][dst] += flow
        self.flow[dst][src] -= flow

    def relabel(self, node: int) -> None:
        minh = math.inf
        for nei in range(self.size):
            if self.remaining_capacity(node, nei) > 0:
                minh = min(minh, self.height[nei])
        if minh < math.inf:
            self.height[node] = minh + 1

    def find_max_height_vertices(self, source: int, sink: int) -> List[int]:
        max_height = []
        for i in range(self.size):
            if i != source and i != sink and self.excess[i] > 0:
                if max_height and self.height[i] > self.height[max_height[0]]:
                    max_height.clear()
                if not max_height or self.height[i] == self.height[max_height[0]]:
                    max_height.append(i)
        return max_height
    
    def main(self, source: int, sink: int) -> int:
        self.build()
        self.height[source] = self.size
        self.excess[source] = math.inf
        for nei in range(self.size):
            if nei == source: continue
            self.push(source, nei)
        while current := self.find_max_height_vertices(source, sink):
            for node in current:
                pushed = False
                for nei in range(self.size):
                    if self.excess[node] == 0: break
                    if self.remaining_capacity(node, nei) > 0 and self.height[node] == self.height[nei] + 1:
                        self.push(node, nei)
                        pushed = True
                if not pushed:
                    self.relabel(node)
                    break
        return sum(self.flow[source])

In [ ]:
"""
Push Relabel Algorithm
- current-arc to avoid iterating over all the edges each time in the discharge operation
- uses adjacency list to avoid the iteration over all the vertices, so only iterate over neighbor vertices
"""
class PushRelabelV3:
    def __init__(self, size: int, edges: List[Tuple[int, int, int]]):
        self.size = size
        self.edges = edges
        self.height = [0] * size
        self.excess = [0] * size
        self.ptr = [0] * self.size
        self.adj_list = [[] for _ in range(self.size)]
        self.cap = [[0] * size for _ in range(size)]
        self.flow = [[0] * size for _ in range(size)]
        self.excess_queue = deque()

    def remaining_capacity(self, src: int, dst: int) -> int:
        return self.cap[src][dst] - self.flow[src][dst]

    def build(self):
        for src, dst, cap in self.edges:
            self.cap[src][dst] += cap
            self.adj_list[src].append(dst)
            self.adj_list[dst].append(src) # residual edge

    def push(self, src: int, dst: int) -> None:
        flow = min(self.excess[src], self.remaining_capacity(src, dst))
        self.excess[src] -= flow
        self.excess[dst] += flow
        self.flow[src][dst] += flow
        self.flow[dst][src] -= flow
        if flow > 0 and self.excess[dst] == flow:
            self.excess_queue.append(dst)

    def relabel(self, node: int) -> None:
        minh = math.inf
        for nei in self.adj_list[node]:
            if self.remaining_capacity(node, nei) > 0:
                minh = min(minh, self.height[nei])
        if minh < math.inf:
            self.height[node] = minh + 1
    
    def discharge(self, node: int) -> None:
        while self.excess[node] > 0:
            # print(self.excess[node], self.ptr[node], self.adj_list[node])
            if self.ptr[node] < len(self.adj_list[node]):
                nei = self.adj_list[node][self.ptr[node]]
                if self.remaining_capacity(node, nei) > 0 and self.height[node] > self.height[nei]:
                    self.push(node, nei)
                else:
                    self.ptr[node] += 1
            else:
                self.relabel(node)
                self.ptr[node] = 0
    
    def main(self, source: int, sink: int) -> int:
        self.build()
        self.height[source] = self.size
        self.excess[source] = math.inf
        for nei in self.adj_list[source]:
            self.push(source, nei)
        while self.excess_queue:
            node = self.excess_queue.popleft()
            if node != source and node != sink:
                self.discharge(node)
        return sum(self.flow[source])

In [121]:
"""
Push Relabel Algorithm
- current-arc to avoid iterating over all the edges each time in the discharge operation
- uses adjacency list to avoid the iteration over all the vertices, so only iterate over neighbor vertices
"""
class PushRelabelV4:
    def __init__(self, size: int, edges: List[Tuple[int, int, int]]):
        self.size = size
        self.edges = edges
        self.height = [0] * size
        self.excess = [0] * size
        self.ptr = [0] * self.size
        self.adj_list = [[] for _ in range(self.size)]
        self.cap = defaultdict(Counter)
        self.flow = defaultdict(Counter)
        self.excess_queue = deque()
        self.relabel_count = 0

    def remaining_capacity(self, src: int, dst: int) -> int:
        return self.cap[src][dst] - self.flow[src][dst]

    def build(self):
        for src, dst, cap in self.edges:
            self.cap[src][dst] += cap
            self.adj_list[src].append(dst)
            self.adj_list[dst].append(src) # residual edge

    def push(self, src: int, dst: int) -> None:
        flow = min(self.excess[src], self.remaining_capacity(src, dst))
        self.excess[src] -= flow
        self.excess[dst] += flow
        self.flow[src][dst] += flow
        self.flow[dst][src] -= flow
        if flow > 0 and self.excess[dst] == flow:
            self.excess_queue.append(dst)

    def relabel(self, node: int) -> None:
        self.relabel_count += 1
        minh = math.inf
        for nei in self.adj_list[node]:
            if self.remaining_capacity(node, nei) > 0:
                minh = min(minh, self.height[nei])
        if minh < math.inf:
            self.height[node] = minh + 1
    
    def discharge(self, node: int) -> None:
        while self.excess[node] > 0:
            # print(self.excess[node], self.ptr[node], self.adj_list[node])
            if self.ptr[node] < len(self.adj_list[node]):
                nei = self.adj_list[node][self.ptr[node]]
                if self.remaining_capacity(node, nei) > 0 and self.height[node] > self.height[nei]:
                    self.push(node, nei)
                else:
                    self.ptr[node] += 1
            else:
                self.relabel(node)
                self.ptr[node] = 0
    
    def main(self, source: int, sink: int) -> int:
        self.build()
        self.height[source] = self.size
        self.excess[source] = math.inf
        for nei in self.adj_list[source]:
            self.push(source, nei)
        while self.excess_queue:
            node = self.excess_queue.popleft()
            if node != source and node != sink:
                self.discharge(node)
        print(f'relabel count: {self.relabel_count}')
        return self.excess[sink]
        return sum(self.flow[source].values())

In [122]:
"""
Push Relabel Algorithm
- current-arc to avoid iterating over all the edges each time in the discharge operation
- uses adjacency list to avoid the iteration over all the vertices, so only iterate over neighbor vertices
- preprocess step where it provides labels from a backwards bfs from sink to the source
- TODO: gap heuristic to reduce useless relabeling => relabel when a node is disconnected from the sink
"""
class PushRelabelV5:
    def __init__(self, size: int, edges: List[Tuple[int, int, int]]):
        self.size = size
        self.edges = edges
        self.excess = [0] * size
        self.ptr = [0] * self.size
        self.adj_list = [[] for _ in range(self.size)]
        self.cap = defaultdict(Counter)
        self.flow = defaultdict(Counter)
        self.excess_queue = deque()
        self.relabel_count = 0

    def remaining_capacity(self, src: int, dst: int) -> int:
        return self.cap[src][dst] - self.flow[src][dst]

    def build(self):
        for src, dst, cap in self.edges:
            self.cap[src][dst] += cap
            self.adj_list[src].append(dst)
            self.adj_list[dst].append(src) # residual edge

    """
    Performs backwards bfs from the sink node to the source node to build up the heights from 0 to n - 1
    """
    def reverse_bfs(self, source: int, sink: int) -> None:
        self.height = [self.size] * self.size
        self.height[sink] = 0
        queue = deque([sink])
        while queue:
            node = queue.popleft()
            for nei in self.adj_list[node]:
                if self.height[nei] == self.size:
                    self.height[nei] = self.height[node] + 1
                    queue.append(nei)
        self.height[source] = self.size

    def push(self, src: int, dst: int) -> None:
        flow = min(self.excess[src], self.remaining_capacity(src, dst))
        self.excess[src] -= flow
        self.excess[dst] += flow
        self.flow[src][dst] += flow
        self.flow[dst][src] -= flow
        if flow > 0 and self.excess[dst] == flow:
            self.excess_queue.append(dst)

    def relabel(self, node: int) -> None:
        self.relabel_count += 1
        minh = math.inf
        for nei in self.adj_list[node]:
            if self.remaining_capacity(node, nei) > 0:
                minh = min(minh, self.height[nei])
        if minh < math.inf:
            self.height[node] = minh + 1
    
    def discharge(self, node: int) -> None:
        while self.excess[node] > 0:
            if self.ptr[node] < len(self.adj_list[node]):
                nei = self.adj_list[node][self.ptr[node]]
                if self.remaining_capacity(node, nei) > 0 and self.height[node] > self.height[nei]:
                    self.push(node, nei)
                else:
                    self.ptr[node] += 1
            else:
                self.relabel(node)
                self.ptr[node] = 0
    
    def main(self, source: int, sink: int) -> int:
        self.build()
        self.reverse_bfs(source, sink)
        self.excess[source] = math.inf
        for nei in self.adj_list[source]:
            self.push(source, nei)
        while self.excess_queue:
            node = self.excess_queue.popleft()
            if node != source and node != sink:
                self.discharge(node)
        print(f'relabel count: {self.relabel_count}')
        return self.excess[sink]
        return sum(self.flow[source].values())

## MPM ALGORITHM

In [123]:
class FlowEdge:
    def __init__(self, src: int, dst: int, cap: int, flow: int = 0):
        self.src = src
        self.dst = dst
        self.cap = cap
        self.flow = 0

    def __repr__(self):
        return f'source node: {self.src}, destination node: {self.dst}, capacity: {self.cap}, flow: {self.flow} ======'

class MPM:
    def __init__(self, size: int, edges: List[Tuple[int, int, int]]):
        self.size = size
        self.edges = edges

    def build(self) -> None:
        self.flowedges = []
        self.adj_list = {}
        for src, dst, cap in self.edges:
            self.flowedges.append(FlowEdge(src, dst, cap))
            if src not in self.adj_list:
                self.adj_list[src] = []
            self.adj_list[src].append(len(self.flowedges) - 1)
            self.flowedges.append(FlowEdge(src, dst, 0)) # residual edge
            if dst not in self.adj_list:
                self.adj_list[dst] = []
            self.adj_list[dst].append(len(self.flowedges) - 1)

    def reset(self) -> None:
        self.potential_in = [0] * self.size
        self.potential_out = [0] * self.size
        self.outbound_edges = defaultdict(dict)
        self.inbound_edges = defaultdict(dict)

    def remaining_capacity(self, index: int) -> int:
        return self.flowedges[index].cap - self.flowedges[index].flow

    def potential(self, node: int) -> int:
        return min(self.potential_in[node], self.potential_out[node])

    def bfs(self, source: int, sink: int) -> bool:
        self.level = [-1] * self.size
        queue = deque([source])
        self.level[source] = 0
        while queue:
            node = queue.popleft()
            for index in self.adj_list[node]:
                nei = self.flowedges[index]
                if self.level[nei.dst] == -1 and self.remaining_capacity(index) > 0:
                    self.level[nei.dst] = self.level[node] + 1
                    queue.append(nei.dst)
        return self.level[sink] != -1

    def remove_node(self, node: int) -> None:
        for index in self.inbound_edges[node]:
            src = self.flowedges[index].src
            self.outbound_edges[src].remove(index)
            self.potential_out[src] -= self.remaining_capacity(index)
        for index in self.outbound_edges[node]:
            dst = self.flowedges[index].dst
            self.inbound_edges[dst].remove(index)
            self.potential_in[dst] -= self.remaining_capacity(index)

    def push(self, src: int, target: int, flow: int, forward: bool) -> None:
        excess = Counter({src: flow})
        queue = deque([src])
        while queue:
            node = queue.popleft()
            if node == target: break
            must_push = excess[node]
            index = 0
            while True:
                try:
                    nei_index = self.outbound_edges[node][index] if forward else self.inbound_edges[node][index]
                    nei_node = self.flowedges[nei_index].dst if forward else self.flowedges[nei_index].src
                except:
                    print('index', index, 'node', node, 'forward', forward, 'flow', flow, 'src', src, 'target', target, 'lenght of flow edges', len(self.flowedges))
                    print('number of outbound edges', len(self.outbound_edges[node]))
                    raise Exception()
                pushed = min(must_push, self.remaining_capacity(nei_index))
                if pushed == 0: break
                if forward:
                    self.potential_out[node] -= pushed
                    self.potential_in[nei_node] -= pushed
                else:
                    self.potential_out[nei_node] -= pushed
                    self.potential_in[node] -= pushed
                if excess[nei_node] == 0:
                    queue.append(nei_node)
                excess[nei_node] += pushed
                self.flowedges[nei_index].flow += pushed
                self.flowedges[nei_index^1].flow -= pushed
                must_push -= pushed
                if self.remaining_capacity(nei_index) == 0:
                    if forward:
                        self.inbound_edges[nei_node][nei_index] -= 1
                        self.outbound_edges[node][nei_index] -= 1
                        if self.inbound_edges[nei_node][nei_index] == 0:
                            self.inbound_edges[nei_node].pop(nei_index)
                        if self.outbound_edges[node][nei_index] == 0:
                            self.outbound_edges[node].pop(nei_index)
                    else:
                        self.outbound_edges[nei_node][nei_index] -= 1
                        self.inbound_edges[node][nei_index] -= 1
                        if self.outbound_edges[nei_node][nei_index] == 0:
                            self.outbound_edges[nei_node].pop(nei_index)
                        if self.inbound_edges[node][nei_index] == 0:
                            self.inbound_edges[node].pop(nei_index)
                    index += 1
                else:
                    break
                if must_push == 0: break

    def main(self, source: int, sink: int) -> int:
        self.build()
        self.alive = [True] * self.size
        maxflow = 0
        while True:
            if not self.bfs(source, sink): break
            self.reset()
            for i in range(len(self.flowedges)):
                if self.remaining_capacity(i) == 0: continue
                src, dst = self.flowedges[i].src, self.flowedges[i].dst
                if self.level[src] + 1 == self.level[dst] and (self.level[dst] < self.level[sink] or dst == sink):
                    if i not in self.outbound_edges[src]:
                        self.outbound_edges[src][i] = 0
                    if i not in self.inbound_edges[dst]:
                        self.inbound_edges[dst][i] = 0
                    self.outbound_edges[src][i] += 1
                    self.inbound_edges[dst][i] += 1
                    self.potential_out[src] += self.remaining_capacity(i)
                    self.potential_in[dst] += self.remaining_capacity(i)
            self.potential_in[source] = self.potential_out[sink] = math.inf
            while True:
                node = -1
                for i in range(self.size):
                    if not self.alive[i]: continue
                    if node == -1 or self.potential(i) < self.potential(node):
                        node = i
                if node == -1: break
                if self.potential(node) == 0: 
                    self.alive[node] = False
                    self.remove_node(node)
                    continue
                flow = self.potential(node)
                maxflow += flow
                self.push(node, source, flow, False)
                self.push(node, sink, flow, True)
                self.alive[node] = False
                self.remove_node(node)
        return maxflow


# Preflow Push-Relabel with scaling factor

In [124]:
"""
Implmentation of the preflow push-relabel scaling algorithm for maximum flow.

Push Relabel Algorithm
- current-arc to avoid iterating over all the edges each time in the discharge operation
- uses adjacency list to avoid the iteration over all the vertices, so only iterate over neighbor vertices
"""
class PushRelabelScale:
    def __init__(self, size: int, edges: List[Tuple[int, int, int]]):
        self.size = size
        self.edges = edges
        self.excess = [0] * self.size
        self.ptr = [0] * self.size # current-arc
        self.levels = [[] for _ in range(2*self.size)]
        self.adj_list = [[] for _ in range(self.size)]
        self.cap = defaultdict(Counter)
        self.flow = defaultdict(Counter)
        self.relabel_count = 0

    def remaining_capacity(self, src: int, dst: int) -> int:
        return self.cap[src][dst] - self.flow[src][dst]

    def build(self):
        self.delta = 0
        for src, dst, cap in self.edges:
            self.cap[src][dst] += cap
            self.delta = max(self.delta, self.cap[src][dst])
            self.adj_list[src].append(dst)
            self.adj_list[dst].append(src) # residual edge
        highest_bit_set = self.delta.bit_length() - 1
        self.delta = 1 << highest_bit_set

    """
    Performs backwards bfs from the sink node to the source node to build up the heights from 0 to n - 1
    """
    def reverse_bfs(self, source: int, sink: int) -> None:
        self.height = [self.size] * self.size
        self.height[sink] = 0
        queue = deque([sink])
        while queue:
            node = queue.popleft()
            for nei in self.adj_list[node]:
                if self.height[nei] == self.size:
                    self.height[nei] = self.height[node] + 1
                    queue.append(nei)
        self.height[source] = self.size

    def init_push(self, src: int, dst: int) -> None:
        flow = min(self.excess[src], self.remaining_capacity(src, dst))
        self.excess[src] -= flow
        self.excess[dst] += flow
        self.flow[src][dst] += flow
        self.flow[dst][src] -= flow

    def push(self, src: int, dst: int, sink: int) -> bool:
        flow = min(self.excess[src], self.remaining_capacity(src, dst), self.delta - self.excess[dst]) if dst != sink else min(self.excess[src], self.remaining_capacity(src, dst))
        # print('src', src, 'dst', dst, 'flow', flow, 'delta', self.delta)
        self.excess[src] -= flow
        self.excess[dst] += flow
        self.flow[src][dst] += flow
        self.flow[dst][src] -= flow
        if self.excess[dst] > self.delta // 2:
            self.levels[self.height[dst]].append(dst)
            return True
        return False

    def relabel(self, node: int) -> None:
        self.relabel_count += 1
        minh = math.inf
        for nei in self.adj_list[node]:
            if self.remaining_capacity(node, nei) > 0:
                minh = min(minh, self.height[nei])
        if minh < math.inf:
            self.height[node] = minh + 1
    
    def discharge(self, node: int, sink: int) -> bool:
        # print(node, 'level', self.height[node])
        found = False
        while self.excess[node] > self.delta // 2:
            # print('node', node, self.excess[node])
            if self.ptr[node] < len(self.adj_list[node]):
                nei = self.adj_list[node][self.ptr[node]]
                # print(node, nei, self.remaining_capacity(node, nei))
                if self.remaining_capacity(node, nei) > 0 and self.height[node] == self.height[nei] + 1:
                    found |= self.push(node, nei, sink)
                else:
                    self.ptr[node] += 1
            else:
                self.relabel(node)
                # print(self.height)
                self.levels[self.height[node]].append(node)
                # print(self.levels)
                self.ptr[node] = 0
                return False
        return found
    
    def main(self, source: int, sink: int) -> int:
        self.build()
        self.reverse_bfs(source, sink)
        self.excess[source] = math.inf
        for nei in self.adj_list[source]:
            self.init_push(source, nei)
        while self.delta > 0:
            level = math.inf
            self.excess[source] = 0
            for i, excess in enumerate(self.excess):
                # print('node', i, 'excess', excess, self.delta)
                if i != source and i != sink and excess > self.delta//2:
                    # print(self.levels)
                    self.levels[self.height[i]].append(i)
                    # print(self.levels)
                    level = min(level, self.height[i])
            # print('level', level, self.levels)
            while level < 2*self.size:
                while self.levels[level]:
                    node = self.levels[level].pop()
                    # print('level', level, node)
                    if node != source and node != sink:
                        found = self.discharge(node, sink)
                        if found: level -= 1
                level += 1
            self.delta >>= 1
        print(f'relabel count: {self.relabel_count}')
        return self.excess[sink]
        return sum(self.flow[source].values())

# [Download Speed](https://cses.fi/problemset/task/1694/)

In [ ]:
urls = ['https://cses.fi/file/acec992e42fe3462f07114ad2d5f7ce9ff27434922e9b52f39006310ca79d019/1/1/', \
    'https://cses.fi/file/558f035a5dce8931e19371bda522b5a81d28a9a1a1835a6205e566ca9de324c8/1/1/', \
    'https://cses.fi/file/9201642e4901d251a2c18f26429a67089a018a07cd3aa6025cb5fd12d4f88126/1/1/', \
    'https://cses.fi/file/654fbbbac2b61ff15187c1d399394ea7e27b05b3dddf32bdba1bb1c6708e3593/1/1/', \
    'https://cses.fi/file/8286fe339a5312417d20620138dec793deb78cd8960f33ddc4f521982e71f046/1/1/', \
    'https://cses.fi/file/297a2fce46a4102cbd86bea796751acd566fcae258aa00b62d34f5436e441b27/1/1/', \
    'https://cses.fi/file/f1cb0fbf03699e8e91a47846d49e084dae8ec899186d7766461383b5bf562452/1/1/', \
    'https://cses.fi/file/d31400a9196af8d78037127201e471353fcd1f5aaecc9939ea4740a054559c0f/1/1/', \
    'https://cses.fi/file/963201f693af2a27f8d43a78a6213b938576971e71fd5270ad62b538cae9cd47/1/1/', \
    'https://cses.fi/file/9b1a8c894a16cc3228c663a38b764156f7f47183b2f7b206866f935d693dbae7/1/1/', \
    'https://cses.fi/file/e27523c04940efd4cddc19cb7ad99a65635c2fb88cf1c86a7492e08089e8c942/1/1/', \
    'https://cses.fi/file/a09e3665a05e05a0f4e6d590b271ba889bed02c5aae69522d38d8bf1c62aa371/1/1/', \
    'https://cses.fi/file/ec19840ed099c8e55fd77bf40b1cf4f6fdbd43c0a63c74dfe736de4d38cb67cd/1/1/']

In [ ]:
"""
Using the dfs implementation as the base case, it was tested to work in the online judge.
"""
results = [0]*len(urls)
for i, url in enumerate(urls):
    data = urllib.heightsrequest.urlopen(url)
    for j, line in enumerate(map(lambda line: line.decode('utf-8').strip('\n'), data)):
        if j == 0:
            n, m = map(int, line.split())
            edges = []
        else:
            u, v, cap = map(int, line.split())
            edges.append((u - 1, v - 1, cap))
    start_time = time.perf_counter()
    mf = FordFulkersonMaxFlowV2(n, edges).main_dfs(0, n - 1)
    end_time = time.perf_counter()
    results[i] = mf
    print(f'Finished testcase: {i} in {end_time - start_time} seconds')

# BENCHMARK

In [120]:
for i, url in enumerate(urls):
    data = urllib.request.urlopen(url)
    for j, line in enumerate(map(lambda line: line.decode('utf-8').strip('\n'), data)):
        if j == 0:
            n, m = map(int, line.split())
            edges = []
        else:
            u, v, cap = map(int, line.split())
            edges.append((u - 1, v - 1, cap))
    # start_time = time.perf_counter()
    # mf_dfs1 = FordFulkersonMaxFlowV1(n, edges).main_dfs(0, n - 1)
    # end_time = time.perf_counter()
    # duration_dfs1 = duration(start_time, end_time)
    # start_time = time.perf_counter()
    # mf_edmonds1 = FordFulkersonMaxFlowV1(n, edges).main_edmonds_karp(0, n - 1)
    # end_time = time.perf_counter()
    # duration_edmonds_karp1 = duration(start_time, end_time)
    # start_time = time.perf_counter()
    # mf_cp_scaling1 = FordFulkersonMaxFlowV1(n, edges).main_capacity_scaling(0, n - 1)
    # end_time = time.perf_counter()
    # duration_cp_scaling1 = duration(start_time, end_time)
    # start_time = time.perf_counter()
    # mf_dinics1 = FordFulkersonMaxFlowV1(n, edges).main_dinics(0, n - 1)
    # end_time = time.perf_counter()
    # duration_dinics1 = duration(start_time, end_time)
    # start_time = time.perf_counter()
    # mf_dfs = FordFulkersonMaxFlowV2(n, edges).main_dfs(0, n - 1)
    # end_time = time.perf_counter()
    # duration_dfs = duration(start_time, end_time)
    # start_time = time.perf_counter()
    # mf_edmonds = FordFulkersonMaxFlowV2(n, edges).main_edmonds_karp(0, n - 1)
    # end_time = time.perf_counter()
    # duration_edmonds_karp = duration(start_time, end_time)
    # start_time = time.perf_counter()
    # mf_cp_scaling = FordFulkersonMaxFlowV2(n, edges).main_capacity_scaling(0, n - 1)
    # end_time = time.perf_counter()
    # duration_cp_scaling = duration(start_time, end_time)
    # start_time = time.perf_counter()159732
    # mf_dinics = FordFulkersonMaxFlowV2(n, edges).main_dinics(0, n - 1)
    # end_time = time.perf_counter()
    # duration_dinics = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_push_relabel = PushRelabelV3(n, edges).main(0, n - 1)
    end_time = time.perf_counter()
    duration_push_relabel = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_push_relabel4 = PushRelabelV4(n, edges).main(0, n - 1)
    end_time = time.perf_counter()
    duration_push_relabel4 = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_push_relabel5 = PushRelabelV5(n, edges).main(0, n - 1)
    end_time = time.perf_counter()
    duration_push_relabel5 = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_push_relabel_scale = PushRelabelScale(n, edges).main(0, n - 1)
    end_time = time.perf_counter()
    duration_push_relabel_scale = duration(start_time, end_time)
    # start_time = time.perf_counter()
    # mf_mpm = MPM(n, edges).main(0, n - 1)
    # end_time = time.perf_counter()
    # duration_mpm = duration(start_time, end_time)
    # assert mf_edmonds1 == results[i], f'Failed on testcase: {i}, output: {mf_edmonds1}, expected: {results[i]} for edmonds-karp algorithm v1'
    # assert mf_cp_scaling1 == results[i], f'Failed on testcase: {i}, output: {mf_cp_scaling1}, expected: {results[i]} for capacity scaling algorithm v1'
    # assert mf_dinics1 == results[i], f'Failed on testcase: {i}, output: {mf_dinics1}, expected: {results[i]} for dinics algorithm v1'
    # assert mf_edmonds == results[i], f'Failed on testcase: {i}, output: {mf_edmonds}, expected: {results[i]} for edmonds-karp algorithm v2'
    # assert mf_cp_scaling == results[i], f'Failed on testcase: {i}, output: {mf_cp_scaling}, expected: {results[i]} for capacity scaling algorithm v2'
    # assert mf_dinics == results[i], f'Failed on testcase: {i}, output: {mf_dinics}, expected: {results[i]} for dinics algorithm v2'
    assert mf_push_relabel == results[i], f'Failed on testcase: {i}, output: {mf_push_relabel}, expected: {results[i]} for push-relabel algorithm v3'
    assert mf_push_relabel4 == results[i], f'Failed on testcase: {i}, output: {mf_push_relabel4}, expected: {results[i]} for push-relabel algorithm v4'
    assert mf_push_relabel5 == results[i], f'Failed on testcase: {i}, output: {mf_push_relabel5}, expected: {results[i]} for push-relabel algorithm v5'
    assert mf_push_relabel_scale == results[i], f'Failed on testcase: {i}, output: {mf_push_relabel_scale}, expected: {results[i]} for push-relabel algorithm scale'
    # assert mf_mpm == results[i], f'Failed on testcase: {i}, output: {mf_mpm}, expected: {results[i]} for mpm algorithm'
    # print(f'dfs v1: {conv_seconds_milliseconds(duration_dfs1)} milliseconds')
    # print(f'edmonds-karp v1: {conv_seconds_milliseconds(duration_edmonds_karp1)} milliseconds')
    # print(f'capacity scaling v1: {conv_seconds_milliseconds(duration_cp_scaling1)} milliseconds')
    # print(f'dinics v1: {conv_seconds_milliseconds(duration_dinics1)} milliseconds')
    # print(f'dfs v2: {conv_seconds_milliseconds(duration_dfs)} milliseconds')
    # print(f'edmonds-karp v2: {conv_seconds_milliseconds(duration_edmonds_karp)} milliseconds')
    # print(f'capacity scaling v2: {conv_seconds_milliseconds(duration_cp_scaling)} milliseconds')
    # print(f'dinics v2: {conv_seconds_milliseconds(duration_dinics)} milliseconds')
    print(f'push-relabel v3: {conv_seconds_milliseconds(duration_push_relabel)} milliseconds')
    print(f'push-relabel v4: {conv_seconds_milliseconds(duration_push_relabel4)} milliseconds')
    print(f'push-relabel v5: {conv_seconds_milliseconds(duration_push_relabel5)} milliseconds')
    print(f'push-relabel scale: {conv_seconds_milliseconds(duration_push_relabel_scale)} milliseconds')
    # print(f'mpm: {conv_seconds_milliseconds(duration_mpm)} milliseconds')
    print(f'============================================================== Test Case {i} Passed ==============================================================')

relabel count: 3
relabel count: 1
relabel count: 1
push-relabel v3: 0.06270499943639152 milliseconds
push-relabel v4: 0.16649800090817735 milliseconds
push-relabel v5: 0.10178899901802652 milliseconds
push-relabel scale: 0.11550400085980073 milliseconds
============================================================== Test Case 0 Passed ==============================================================
relabel count: 3
relabel count: 0
relabel count: 0
push-relabel v3: 0.3314440000394825 milliseconds
push-relabel v4: 0.6361579980875831 milliseconds
push-relabel v5: 0.34792199949151836 milliseconds
push-relabel scale: 0.4055989993503317 milliseconds
============================================================== Test Case 1 Passed ==============================================================
relabel count: 3
relabel count: 0
relabel count: 0
push-relabel v3: 0.06519000089610927 milliseconds
push-relabel v4: 0.16359800065401942 milliseconds
push-relabel v5: 0.07399199967039749 milliseconds
push

KeyboardInterrupt: 

# UNIT TESTING

## Conclusion 
Dinics v2 has the fastest runtime in most cases